# TAMU Geography Texas County Contest

This notebook calculates/creates the following items for the county contest:
- number of counties visited per year of Texas residence
- the greatest distance between counties visited (based on centroids)
- the longest contiguous boundary made from the counties visited
- the boldest mile: number of miles driven multiplied by the age of the vehicle
- alphabet bingo: a list of the unique first letters of the counties visited
- a list of all the counties visited
- a map visualizing the counties visited, their contiguous boundary, your travel route(s), the greatest distance between counties

This is all done by intersecting official TxDOT county boundaries (stored in the `data` folder) with kmz(s) of your travel route(s).
These travel routes can be downloaded from the [My Maps](https://www.google.com/maps/d/) site created by Google. More detailed instructions on downloading these routes can be found on the `README.md` file (included) or online.

Example kmz routes can be found in the `example_data` folder. Copy and paste these files to the `data` folder if you want to see an example.

If you do not wish to use the kmz files, you can simply list the counties you visited in the cell below. The code will automatically find the TxDOT boundaries with the same name. Be sure to use the correct spelling, (including capitalizing the first letter) and separating each name with a comma. 

If you use the kmz routes AND list counties, the code will add whatever counties are listed that arent already intersected by the routes. 



# Enter your information in the cell below

This is the only cell where you will have to type things in. Only enter information for the varabiles in ALL_CAPITAL_LETTERS. Be sure to add you info in between the double quotes for each variable.

Once you have entered your info, just hit the "Run All" button at the top of the screen. 

If the cell labeled "Check User Input" says "the following county names can't be found..." then check your spelling against the county names printed, fix any errors and hit "Run All" again. Do this until you no longer get errors.

In [ ]:
# Enter your information below. For each line, enter your info between the double quotes

# How long have you resided in Texas (in years)?
YEARS_OF_RESIDENCE = ""

# How old is your vehicle (in years)? Enter 0 if not applicable
VEHICLE_AGE = ""

# List counties below if you are not using kmz routes OR you want to add counties not intersected by your kmz routes.
# Add the county names between the double quotes, each separated by a comma and with the first letter capitalized
# ex) "Brazos, Burleson, Harris"
USER_COUNTIES = ""

## Check user input

In [ ]:
import geopandas as gpd
import fiona
fiona.supported_drivers["LIBKML"] = "raw"

from txtravel_funcs import check_kmz, check_county_names, load_routes, intersecting_counties
from txtravel_funcs import CountyProcessor

In [ ]:
# Cast user input
if YEARS_OF_RESIDENCE:
    YEARS_OF_RESIDENCE = int(YEARS_OF_RESIDENCE)
else:
    raise ValueError("Please enter a value for YEARS_OF_RESIDENCE and re-run")

if VEHICLE_AGE:
    VEHICLE_AGE = int(VEHICLE_AGE)
else:
    raise ValueError("Please enter a value for VEHICLE_AGE and re-run. Set to 0 if not applicable.")

# Print user input
print(f"Years of Texas residence: {YEARS_OF_RESIDENCE}")
print(f"Age of vehicle: {VEHICLE_AGE}")

# Check for kmz routes
kmz_files = check_kmz()
print("")

# Print listed counties
if USER_COUNTIES:
    county_list = USER_COUNTIES.replace(", ", ",").split(",")
else:
    county_list = []
print(f"Listed counties: {county_list}")

# Open counties and prep TX boundary for later
counties = gpd.read_file("data/Texas_County_Boundaries.geojson")
tx_boundary = counties.dissolve()

# Get UTM crs for Texas for calcualtions later
tx_utm = tx_boundary.estimate_utm_crs()

# Check for valid county names
check_county_names(counties, county_list)

## Open kmz routes and get intersecting counties

In [ ]:
# Create empty variable if user does not add routes
clipped_routes = gpd.GeoDataFrame()

if kmz_files:
    # Load in the kmz routes
    routes = load_routes(kmz_files)

    # Clip to TX Boundary
    clipped_routes = routes.clip(tx_boundary)

    # Add intersetcted counties to the listed counties
    itx_county_names = intersecting_counties(clipped_routes, counties)

    # Append intersecting counties to the county_list
    [county_list.append(name) for name in itx_county_names if name not in county_list]
    county_list = sorted(county_list)

visited_counties = counties[counties["CNTY_NM"].isin(county_list)]

if visited_counties.empty:
    raise ValueError("No counties provided. Please either list counties in the first cell or add kmz routes to the `data` folder")

# Calculate values

In [ ]:
county_processor = CountyProcessor(visited_counties, clipped_routes, YEARS_OF_RESIDENCE, VEHICLE_AGE, tx_utm)

county_count, year_of_res = county_processor.counties_per_yr_tx_res()
greatest_dist = county_processor.greatest_distance_between_counties()
longest_boundary = county_processor.longest_boundary()
distance_driven, vehicle_age = county_processor.boldest_mile()


# Print values and show map

In [ ]:
print(f"You visited {county_count} counties with {year_of_res} years of TX residence. Your ratio is: {round(county_count/year_of_res, 2)}")
print(f"Your greatest distance between county centroids this summer was {round(greatest_dist.loc[0, 'geometry'].length/1609, 2)} miles. See map below.")
print(f"The longest contiguous boundary of counties you visited was {round(longest_boundary.geometry.exterior.length[0][0]/1609, 2)} miles")

print(f"You drove {round(distance_driven, 2)} miles with a {vehicle_age} year old vehicle. Your boldest mile score is {round(distance_driven*vehicle_age, 2)}")


# Create map
m = visited_counties.explore()
longest_boundary.explore(m=m, style_kwds={"fill":False, "weight":5})
if not clipped_routes.empty:
    clipped_routes.explore(m=m, color="blue")
greatest_dist.explore(m=m, color="black", style_kwds={"weight":3})